# Model 1_2
## Dataset: Gym

## Programmer: Giovanni Vecchione
## Date: 4/17/24
## Subject: Machine Learning 2 - Project 6
Use Reinforced Learning (RL) to build the project. Submit your project as Jupyter notebook.

NOTE: Using Q-Learning to the CartPole Enviornment / Not complete

In [20]:
import matplotlib as mtp
import torch
import numpy as np
import gym
import random

In [21]:
#Checks if GPU is being used
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the GPU
    print("Using GPU:", torch.cuda.get_device_name(0)) 
else:
    device = torch.device("cpu")  # Fallback to CPU
    print("GPU not available, using CPU.")

#Using GPU: NVIDIA GeForce GTX 1660 SUPER - Successful
#NOTE: This took some time to set up by installing and pathing the cuda toolkit v.12.4 and the right supplemental packages. This drastically improved
#training time

Using GPU: NVIDIA GeForce GTX 1660 SUPER


In [22]:
#Creates the environment to simulate the world needed. This includes all the necessary physics, rules, goals, etc.
env = gym.make("CartPole-v1")

The env variable now acts as the way you'll interact with the environment. You'll use functions provided by it to:
env.reset(): Get the initial state of the environment.
env.step(action): Take an action and see the resulting new state, reward, and whether the episode is finished.
env.render(): (Optional) Visualize the environment.

In [23]:
print(env.observation_space)  # What the agent observes
print(env.action_space)       # Actions the agent can take

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Discrete(2)


# Using Q-Learning

### *Key Points*
* Q-value Represents the expected future reward the agent can get by taking a particular action in a given state.

## *Setting up a Q-table*
In CartPole, the agent's observations (position, velocity, angle, etc.) are continuous values. To use Q-learning effectively, we'll need to discretize these observations into a finite set of states. Use techniques like binning or clustering to do this.

1. Understanding Discrete States

    * In CartPole, the agent's observations (position, velocity, angle, etc.) are continuous values. To use Q-learning effectively, we'll need to discretize these observations into a finite set of states. You might use techniques like binning or clustering to do this.

2. Initializing the Q-table
    * We'll use a simple data structure like a NumPy array or a Python dictionary to represent our Q-table

3. Interpretation

    * Rows: Each row in the Q-table corresponds to a specific discrete state of the environment.
    * Columns: Each column represents a possible action the agent can take (in CartPole, move left or move right).
    * Values: The value inside a cell Q_table[state, action] represents the estimated future reward the agent expects to get if it takes the action in the state and follows its optimal policy from there onwards.


In [24]:
# Set up Q-table (Starting with 50 as size)
num_states = 50
num_actions = env.action_space.n
Q_table = np.zeros((num_states, num_actions))

#Creates a Q-table with the number of rows equal to the number of discretized states and 
#the number of columns equal to the number of actions. We initialize all the Q-values to zero.


## *Agent Initialization and The Learning Loop*
Create an agent that will interact with the environment to learn the optimal balancing policy.

### *The Agent's Components:*

* The Q-table:  This data structure embodies the core knowledge of the agent. It stores the agent's understanding of the best actions to take in different states.

* Policy: As the Q-table is updated, the agent implicitly develops a policy. This policy is essentially the way it decides which action to take based on its current state. In Q-learning, the policy is often an epsilon-greedy policy, striking a balance between exploring new actions and exploiting its current knowledge.

* Learning Algorithm: The Q-learning update rule itself is the core mechanism by which the agent learns. This algorithm updates the Q-table based on the agent's experiences, progressively improving the agent's knowledge of good and bad actions.

### So, where's the agent?

The agent in Q-learning is an emergent entity. It's not explicitly programmed, but rather emerges from the combination of the Q-table, its implicit policy, and the learning algorithm.

* Discount factor (gamma): Determines how much we value future rewards compared to immediate rewards.
* Learning rate (alpha): Controls how much we update our Q-values with each new experience.

In [25]:
# Hyperparameters
alpha = 0.1
gamma = 0.95
epsilon = 1.0  # Starting with high exploration (1.0)

In [26]:
observation, reward, done, info = env.reset()
action = env.action_space.sample()
print(env.step(action))

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
# Training loop
episodes = 1000
for episode in range(episodes):
    state, info = env.reset()
    done = False

    while not done:
        if random.random() < epsilon: 
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(Q_table[state, :])  # Exploit learned values

        new_state, reward, done, info = env.step(action) # I believe this needs to be changed - Needs a system for rewards etc.

        # Update Q-table
        old_value = Q_table[state, action]
        next_max = np.max(Q_table[new_state, :])  
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        Q_table[state, action] = new_value

        state = new_state

    # Reduce exploration rate over time
    if epsilon > 0.01:
        epsilon *= 0.995

ValueError: too many values to unpack (expected 4)

In [ ]:
# Function sees how it performs after training
for _ in range(5):
  state, info = env.reset()
  done = False
  steps = 0
  while not done:
    action = np.argmax(Q_table[state, :]) 
    state, reward, done, info = env.step(action)
    env.render()
    steps += 1
  print("Episode lasted", steps, "steps")

env.close()

IndexError: arrays used as indices must be of integer (or boolean) type